In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from collections import Counter
import pickle
from datetime import datetime

In [2]:
def read_file(dataset_en, dataset_fr):
    #read the files
    with open(dataset_en) as f:
        sentences_en = [l.strip() for l in f.readlines()]
    
    with open(dataset_fr) as f:
        sentences_fr = [l.strip() for l in f.readlines()]

    sentence_list_en = []
    sentence_list_fr = []
    
    no_sentences = len(sentences_en)
    
    for s in range(no_sentences):
        
        sent_en = sentences_en[s]
        sent_fr = sentences_fr[s]
        
        split_sent_en = sent_en.split()
        split_sent_fr = sent_fr.split()
        
        if len(split_sent_en) <= 50 and len(split_sent_fr) <= 50:
            
            sentence_list_en.append(split_sent_en)
            sentence_list_fr.append(split_sent_fr)

    return sentence_list_en, sentence_list_fr

In [ ]:
sentences_en, sentences_fr = read_file('europarl/training.en','europarl/training.fr')

with open('europarl_split.en.pickle', 'wb') as f:
    pickle.dump(sentences_en, f)
    
with open('europarl_split.fr.pickle', 'wb') as f:
    pickle.dump(sentences_fr, f)

In [5]:
def train_sg_model(sentences, label, size, window, negative, iterations):
    
    #https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
    #https://radimrehurek.com/gensim/models/word2vec.html#id6
        
    # define training data
    model = Word2Vec(sentences, size=size, window=window, min_count = 2, sg = 1, negative =negative, iter=iterations, workers=4)
      
    ext = label + '_' + str(size) + '_' +  str(window)+ '_'  + str(negative)+ '_' +  str(iterations)
    
    model.wv.save_word2vec_format('model_'+ext+'.bin')
    
    # summarize the model
    print(model)
    
    # obtain the vocabulary
    words = list(model.wv.vocab)
    
    with open('words.'+ext+'.pickle', 'wb') as f:
        pickle.dump(words,f)
        
    return model, words

In [8]:
#no of sentences 1968355
with open('europarl_split.en.pickle', 'rb') as f:
    sentences_en = pickle.load(f)

In [11]:
# size = [100,300]
# window = [2,5]
# neg_samples = [5,20]
# iterations = [10,20]

size = [300]
window = [5]
neg_samples = [5]
iterations = [10,20]


for s in size:
    for w in window:
        for n in neg_samples:
            for i in iterations:

                print(s,w,n,i)
                ext = str(s) + '_' + str(w)+ '_'  + str(n)+ '_' +  str(i)
    
                then = datetime.now()
                model_en, words_en = train_sg_model(sentences_en, label = 'en', size=s, window=w, negative=n, iterations=i)
                    
                now = datetime.now()

                print('English skipgram',ext, now-then)

300 5 5 10
Word2Vec(vocab=77338, size=300, alpha=0.025)
English skipgram 300_5_5_10 0:29:21.148644
300 5 5 20
Word2Vec(vocab=77338, size=300, alpha=0.025)
English skipgram 300_5_5_20 0:56:47.911805


In [2]:
model_en = KeyedVectors.load_word2vec_format('model_en.bin')
print(model_en)